In [1]:
library(remotes)
install_version("vctrs", version = "0.5.0", repos = "http://cran.us.r-project.org")
install_version("stringr", version = "1.4.0", repos = "http://cran.us.r-project.org")

In [2]:
library(stringr)

In [3]:
library(anndata)
library(Signac)
library(Seurat)
library(tidyverse)


Attaching SeuratObject

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ forcats 0.5.2 
✔ readr   2.1.3      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()
✖ readr::read_csv() masks anndata::read_csv()
✖ purrr::reduce()   masks Signac::reduce()


In [4]:
seurat <- readRDS("/Users/dominik.klein/data/E14-E15_adata_filtered_seurat_merged_peaks.rds")

In [5]:
new_annotations <- read.csv("/Users/dominik.klein/paper_writing/moscot/endocrine_refinement.csv")

In [6]:
adapt_obs_names <- function(x){
    tmp <- str_split(x, "-")[[1]]
    if(tmp[3] == "0"){
        prefix <- "E14-5"
        }
    else{
        prefix <- "E15-5"
        }
    return(paste(prefix, "_", tmp[1], "-", tmp[2], sep = ""))
    }

In [7]:
new_annotations$obs_name_adapted <- sapply(new_annotations$X, adapt_obs_names)

In [8]:
seurat <- seurat[,new_annotations$obs_name_adapted]

In [9]:
seurat

An object of class Seurat 
475329 features across 16918 samples within 3 assays 
Active assay: ATAC_merged (228259 features, 228259 variable features)
 2 other assays present: RNA, ATAC
 1 dimensional reduction calculated: lsi

In [10]:
seurat@meta.data$rowname <- rownames(seurat@meta.data)

In [11]:
merged_obs <- merge(seurat@meta.data, new_annotations, by.x="rowname", by.y="obs_name_adapted")

In [12]:
seurat@meta.data <- merged_obs

In [13]:
Idents(seurat) <- seurat@meta.data$refinement

In [14]:
Idents(seurat) <- seurat@meta.data$refinement

In [15]:
celltypes_to_keep = c('Ngn3 low', 'Ngn3 high cycling', 'Ngn3 high,0', 'Ngn3 high,1', 'Eps. progenitors', 'Fev+', 'Fev+ Alpha', 
                   'Fev+ Beta', 'Fev+ Delta,0', 'Fev+ Delta,1', 'Alpha', 'Beta', 'Delta', 'Epsilon')

In [16]:
seurat_filtered <- subset(x=seurat, idents= celltypes_to_keep)


In [17]:
seurat_filtered@meta.data <- seurat@meta.data[seurat@meta.data$refinement %in% celltypes_to_keep,]

In [19]:
saveRDS(seurat_filtered, file="/Users/dominik.klein/data/seurat_new_annotations.rds")